# NonlinearModel
* Uses LightGBM to estimate a given (user, item) prediction

In [1]:
import NBInclude: @nbinclude
@nbinclude("TreeModelBase.ipynb")
@nbinclude("SuppressImplicit.ipynb");

┌ Info: lib_lightgbm found in system dirs!
└ @ LightGBM /Users/kundan/.julia/packages/LightGBM/A7zVd/src/LightGBM.jl:28


In [2]:
implicit = false;
linear_alphas = ["LinearExplicit", "LinearImplicit"]
all_features = [
    explicit_raw_alphas
    implicit_raw_alphas
    nondirectional_raw_alphas
    linear_alphas
];

In [3]:
train_model(
    all_features,
    ["LinearExplicit"],
    implicit,
    ["validation"],
    "NonlinearExplicit",
    false;
    λ = [-1.0, 3.5234375, 2.53125, -1.703125, 2.3046875],
);

Progress: 100%|███████████████████████████| Time: 0:00:00 (80.56 ns/it)


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.125046 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3570
[LightGBM] [Info] Number of data points in the train set: 22508916, number of used features: 14
[LightGBM] [Info] Start training from score -0.000794
Iteration: 1, test_1's l2: 1.3295229592964781
Iteration: 2, test_1's l2: 1.328635794087175
Iteration: 3, test_1's l2: 1.3277986450973185
Iteration: 4, test_1's l2: 1.3270492848571478
Iteration: 5, test_1's l2: 1.326313646076653
Iteration: 6, test_1's l2: 1.3256347312225545
Iteration: 7, test_1's l2: 1.3249997418702388
Iteration: 8, test_1's l2: 1.3243737420393922
Iteration: 9, test_1's l2: 1.3237927296021235
Iteration: 10, test_1's l2: 1.3232195413753944
Iteration: 11, test_1's l2: 1.322670900045905
Iteration: 12, test_1's l2: 1.322183504829977
Iteration: 13, test_1's l2: 1.321711

[ Info: 20220709 12:00:18 Saving model...
Progress: 100%|█████████████████████████████████████████| Time: 0:22:19
[ Info: 20220709 12:22:55 Average model value: 0.063294396
[ Info: 20220709 12:22:55 Average model absolute value: 0.17932947
Progress: 100%|███████████████████████████| Time: 0:00:14 ( 0.51 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:01 ( 0.56 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:01 ( 0.55 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:05 ( 0.48 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 ( 0.55 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 ( 0.62 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:02 ( 0.46 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:02 ( 0.46 μs/it)


In [4]:
# 1.30429808358728

In [5]:
function save_final_explicit_model()
    linear_splits = reduce(
        cat,
        [
            read_alpha("LinearExplicit", split, implicit) for
            split in all_nontraining_raw_splits
        ],
    )
    nonlinear_splits = reduce(
        cat,
        [
            read_alpha("NonlinearExplicit", split, implicit) for
            split in all_nontraining_raw_splits
        ],
    )
    sparse_preds = sparse(
        linear_splits.user,
        linear_splits.item,
        linear_splits.rating + nonlinear_splits.rating,
    )
    write_alpha(sparse_preds, [], implicit, "Explicit"; log_test_split = true)
    write_params(Dict("alphas" => ["LinearExplicit", "NonlinearExplicit"]), "Explicit")
end;

In [6]:
save_final_explicit_model()

Progress: 100%|███████████████████████████| Time: 0:00:01 ( 0.61 μs/it)
[ Info: 20220709 12:23:39 validation loss: 1.2850029791372957, β: [1.0000545519606285]
Progress: 100%|███████████████████████████| Time: 0:00:13 ( 0.49 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (10.39 ns/it)
[ Info: 20220709 12:23:55 training loss: 67.37459822571184, β: [1.0000545519606285]
Progress: 100%|███████████████████████████| Time: 0:00:01 ( 0.53 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (30.92 ns/it)
[ Info: 20220709 12:23:57 test loss: 1.3138729632072033, β: [1.0000545519606285]
Progress: 100%|███████████████████████████| Time: 0:00:14 ( 0.50 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:01 ( 0.54 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:01 ( 0.53 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:05 ( 0.49 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 ( 0.55 μs/it)
Progress: 100%|█████████████